In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3

## Creating the pipeData table

In [3]:
# Load the CSV
TeAllPointsFile = "../resources/TeAllPoints.csv"
pointsdf = pd.read_csv(TeAllPointsFile, encoding="ISO-8859-1")

# Loc column has leading spaces, so we convert to remove them
pointsdf.Loc = pd.to_numeric(pointsdf['Loc'], errors='coerce')

# pick the columns for the table
pipeDataTable = pd.DataFrame({
    'loc': pointsdf['Loc'],
    'locName': pointsdf['Loc Name'],
    'locState': pointsdf['Loc St Abbrev'],
    'locCounty': pointsdf['Loc Cnty'],
    'locZone': pointsdf['Loc Zone'],
    'dirFlo': pointsdf['Dir Flo'],
    'locType': pointsdf['Loc Type Ind']
})

pipeDataTable.head(1)

,loc,locName,locState,locCounty,locZone,dirFlo,locType
0,70059,"Chevron - Perth Amboy, NJ",NJ,MIDDLESEX,3,Delivery,UNK


## Creating the dailyFlo table

In [4]:
# Load the XLSX
flowsFile = "../resources/flows_last10.xlsx"
flowsdf = pd.concat(pd.read_excel(flowsFile, sheet_name=None), ignore_index=True)

dailyFloTable = pd.DataFrame({
    'loc': flowsdf.Loc,
    'gas_date': flowsdf.Eff_Gas_Day,
    'Total_Design_Capacity': flowsdf.Total_Design_Capacity,
    'Operating_Capacity': flowsdf.Operating_Capacity,
    'Total_Scheduled_Quantity': flowsdf.Total_Scheduled_Quantity,
    'TSP_Name': flowsdf.TSP_Name
})

dailyFloTable.head(1)

,loc,gas_date,Total_Design_Capacity,Operating_Capacity,Total_Scheduled_Quantity,TSP_Name
0,70004,2020-08-21,593830,600490,0,TX EAST TRAN


## Sending data to the DB

In [5]:
conn = sqlite3.connect('ng_db_p2.sqlite')
c = conn.cursor()

In [6]:
c.execute('''CREATE TABLE pipeData
            ([loc] INTEGER PRIMARY KEY,
            [locName] TEXT,
            [locState] TEXT,
            [locCounty] TEXT,
            [locZone] TEXT,
            [dirFlo] TEXT,
            [locType] TEXT)''')

In [7]:
c.execute('''CREATE TABLE dailyFlo
          ([loc] INTEGER,
           [gas_date] date,
           [Total_Design_Capacity] INTEGER,
           [Operating_Capacity] INTEGER,
           [Total_Scheduled_Quantity] INTEGER,
           [TSP_Name] TEXT,
           PRIMARY KEY(loc,gas_date))''')

In [8]:
pipeDataTable.to_sql('pipeData', conn, if_exists='append', index = False)

In [9]:
dailyFloTable.to_sql('dailyFlo', conn, if_exists='append', index = False)

In [34]:
c.execute("SELECT * FROM pipeData")

rows = c.fetchall()

for row in rows:
    print(row)

(70004, 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
(70011, 'COLUMBIA GAS OF  PA.  - EAGLE, PA', 'PA', 'CHESTER', '3', 'D', 'INT')
(70017, 'INDIANA GAS - GREENSBURG, IN', 'IN', 'DECATUR', '2', 'D', 'LDC')
(70018, 'INDIANA GAS - SEYMOUR, IN', 'IN', 'JACKSON', '2', 'D', 'LDC')
(70020, 'EQUITRANS - WAYNESBURG, PA', 'PA', 'GREENE', '2', 'D', 'INT')
(70030, 'PHILADELPHIA GAS WORKS - POINT BREEZE,PA', 'PA', 'PHILADELPHIA', '3', 'D', 'LDC')
(70033, 'PECO ENERGY - CROMBY GENERATION, PA', 'PA', 'CHESTER', '3', 'D', 'LDC')
(70034, 'PHILADELPHIA GAS WORKS - LEVICK ST., PA', 'PA', 'MONTGOMERY', '3', 'D', 'LDC')
(70035, 'PECO ENERGY - TILGHMAN ST. - CHESTER, PA', 'PA', 'DELAWARE', '3', 'D', 'LDC')
(70036, 'PECO ENERGY - WEST CONSHOHOCKEN, PA', 'PA', 'MONTGOMERY', '3', 'D', 'LDC')
(70039, 'UNITED CITIES - HARRISBURG, IL', 'IL', 'SALINE', '1', 'D', 'LDC')
(70040, 'Central Il. - New Dennison, IL', 'IL', 'WILLIAMSON', '1', 'D', 'UNK')
(70041, 'COLUMBIA GAS - L

In [10]:
c.execute("SELECT * FROM dailyFlo")

rows = c.fetchall()

for row in rows:
    print(row)

(70004, '2020-08-21 00:00:00', 593830, 600490, 0, 'TX EAST TRAN')
(70011, '2020-08-21 00:00:00', 433289, 444405, 124469, 'TX EAST TRAN')
(70017, '2020-08-21 00:00:00', 21573, 21573, 160, 'TX EAST TRAN')
(70018, '2020-08-21 00:00:00', 57331, 58910, 249, 'TX EAST TRAN')
(70020, '2020-08-21 00:00:00', 5011, 5488, 0, 'TX EAST TRAN')
(70030, '2020-08-21 00:00:00', 221946, 222548, 37960, 'TX EAST TRAN')
(70033, '2020-08-21 00:00:00', 123798, 119263, 0, 'TX EAST TRAN')
(70034, '2020-08-21 00:00:00', 288841, 287378, 1731, 'TX EAST TRAN')
(70035, '2020-08-21 00:00:00', 249094, 249434, 0, 'TX EAST TRAN')
(70036, '2020-08-21 00:00:00', 241034, 246546, 61291, 'TX EAST TRAN')
(70039, '2020-08-21 00:00:00', 20809, 20738, 1500, 'TX EAST TRAN')
(70040, '2020-08-21 00:00:00', 63989, 63989, 0, 'TX EAST TRAN')
(70041, '2020-08-21 00:00:00', 248781, 248781, 26707, 'TX EAST TRAN')
(70042, '2020-08-21 00:00:00', 63226, 63226, 6891, 'TX EAST TRAN')
(70043, '2020-08-21 00:00:00', 10692, 10692, 0, 'TX EAST TRA

(73919, '2020-08-22 00:00:00', 305100, 305100, 290000, 'TX EAST TRAN')
(73920, '2020-08-22 00:00:00', 508500, 508500, 38750, 'TX EAST TRAN')
(73921, '2020-08-22 00:00:00', 254427, 254427, 123718, 'TX EAST TRAN')
(73923, '2020-08-22 00:00:00', 10177, 10177, 0, 'TX EAST TRAN')
(73924, '2020-08-22 00:00:00', 508500, 508500, 64342, 'TX EAST TRAN')
(73925, '2020-08-22 00:00:00', 610625, 610625, 23000, 'TX EAST TRAN')
(73927, '2020-08-22 00:00:00', 152550, 152550, 110486, 'TX EAST TRAN')
(73930, '2020-08-22 00:00:00', 2035, 2035, 600, 'TX EAST TRAN')
(74040, '2020-08-22 00:00:00', 277877, 277877, 0, 'TX EAST TRAN')
(74058, '2020-08-22 00:00:00', 660131, 660131, 0, 'TX EAST TRAN')
(74059, '2020-08-22 00:00:00', 183860, 183860, 0, 'TX EAST TRAN')
(74128, '2020-08-22 00:00:00', 574801, 574801, 0, 'TX EAST TRAN')
(74186, '2020-08-22 00:00:00', 136041, 136041, 0, 'TX EAST TRAN')
(74187, '2020-08-22 00:00:00', 366393, 366393, 16268, 'TX EAST TRAN')
(74200, '2020-08-22 00:00:00', 661700, 661700, 0,

(71616, '2020-08-25 00:00:00', 280765, 280765, 1051, 'TX EAST TRAN')
(71655, '2020-08-25 00:00:00', 694652, 694652, 262, 'TX EAST TRAN')
(71671, '2020-08-25 00:00:00', 76328, 76328, 0, 'TX EAST TRAN')
(71682, '2020-08-25 00:00:00', 12212, 12212, 0, 'TX EAST TRAN')
(71704, '2020-08-25 00:00:00', 37177, 37177, 0, 'TX EAST TRAN')
(71706, '2020-08-25 00:00:00', 16726, 16726, 336, 'TX EAST TRAN')
(71709, '2020-08-25 00:00:00', 5085, 5085, 0, 'TX EAST TRAN')
(71716, '2020-08-25 00:00:00', 266391, 266391, 0, 'TX EAST TRAN')
(71744, '2020-08-25 00:00:00', 377137, 377137, 139500, 'TX EAST TRAN')
(71745, '2020-08-25 00:00:00', 736372, 736372, 65464, 'TX EAST TRAN')
(71750, '2020-08-25 00:00:00', 124061, 124061, 0, 'TX EAST TRAN')
(71773, '2020-08-25 00:00:00', 4427, 4427, 1, 'TX EAST TRAN')
(71814, '2020-08-25 00:00:00', 65562, 65562, 0, 'TX EAST TRAN')
(71825, '2020-08-25 00:00:00', 71450, 71450, 0, 'TX EAST TRAN')
(71841, '2020-08-25 00:00:00', 16275, 16275, 0, 'TX EAST TRAN')
(71862, '2020-08

(73339, '2020-08-27 00:00:00', 267711, 267711, 116996, 'TX EAST TRAN')
(73341, '2020-08-27 00:00:00', 4046, 4046, 1, 'TX EAST TRAN')
(73345, '2020-08-27 00:00:00', 36172, 36172, 0, 'TX EAST TRAN')
(73346, '2020-08-27 00:00:00', 230707, 230707, 114493, 'TX EAST TRAN')
(73348, '2020-08-27 00:00:00', 16248, 16248, 5, 'TX EAST TRAN')
(73349, '2020-08-27 00:00:00', 207092, 207092, 207649, 'TX EAST TRAN')
(73350, '2020-08-27 00:00:00', 55873, 55873, 1370, 'TX EAST TRAN')
(73355, '2020-08-27 00:00:00', 3328, 3328, 48, 'TX EAST TRAN')
(73362, '2020-08-27 00:00:00', 9037, 9037, 1, 'TX EAST TRAN')
(73364, '2020-08-27 00:00:00', 3896, 3896, 0, 'TX EAST TRAN')
(73379, '2020-08-27 00:00:00', 607831, 607831, 0, 'TX EAST TRAN')
(73385, '2020-08-27 00:00:00', 9005, 9005, 0, 'TX EAST TRAN')
(73388, '2020-08-27 00:00:00', 8879, 8879, 190, 'TX EAST TRAN')
(73391, '2020-08-27 00:00:00', 9732, 9732, 194, 'TX EAST TRAN')
(73393, '2020-08-27 00:00:00', 4496, 4496, 0, 'TX EAST TRAN')
(73399, '2020-08-27 00:00

(73277, '2020-08-29 00:00:00', 3750, 3750, 0, 'TX EAST TRAN')
(73280, '2020-08-29 00:00:00', 156993, 156993, 0, 'TX EAST TRAN')
(73282, '2020-08-29 00:00:00', 15122, 15122, 400, 'TX EAST TRAN')
(73283, '2020-08-29 00:00:00', 36498, 36498, 606, 'TX EAST TRAN')
(73285, '2020-08-29 00:00:00', 9027, 9027, 0, 'TX EAST TRAN')
(73294, '2020-08-29 00:00:00', 105842, 131859, 75802, 'TX EAST TRAN')
(73295, '2020-08-29 00:00:00', 15122, 15122, 200, 'TX EAST TRAN')
(73297, '2020-08-29 00:00:00', 165261, 165261, 78000, 'TX EAST TRAN')
(73298, '2020-08-29 00:00:00', 224117, 224117, 0, 'TX EAST TRAN')
(73304, '2020-08-29 00:00:00', 66510, 66510, 10000, 'TX EAST TRAN')
(73306, '2020-08-29 00:00:00', 248362, 248362, 125000, 'TX EAST TRAN')
(73310, '2020-08-29 00:00:00', 124219, 124219, 5000, 'TX EAST TRAN')
(73311, '2020-08-29 00:00:00', 8612, 8612, 0, 'TX EAST TRAN')
(73313, '2020-08-29 00:00:00', 4878, 4878, 0, 'TX EAST TRAN')
(73316, '2020-08-29 00:00:00', 34856, 34856, 0, 'TX EAST TRAN')
(73318, '2